<a href="https://colab.research.google.com/github/vnirwan/Chatbot-trying/blob/main/fine_tune_VN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introduction
This is a demonstration notebook that shows how to fine-tune a gpt-3 model. Using its "ada" model because it is the cheapest and fastest option available.

The dataset used can be found at the following link: https://www.kaggle.com/datasets/aemreusta/paraphrased-articles-using-gpt3?resource=download.

In order to keep costs low, only 10 data points were used for training and 3 data points were used for validation.

The API key for this notebook is currently hardcoded in the notebook because it is unable to read it from a file or a variable. However, a solution to encrypt the key may be found later.

One limitation to note is that this fine-tuning job was queued for approximately 2 hours and was interrupted multiple times.

Dollar Used: 0.02

In [ ]:
import pandas as pd
import openai
import os
import openai
import getpass

In [ ]:
!pip install --quiet openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 KB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 KB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')

Enter OpenAI API key:··········


In [ ]:
df_raw=pd.read_csv('paraphrased_articles.csv')

In [ ]:
#Take only two columns: abstract and ParaphrasedAbstract
df_raw=df_raw[['Abstract','ParaphrasedAbstract']]
#df.head()

##Data preparation
Data preparation according to the gpt-3 models. Need to define "prompt" and "completion" and convert data to JSNOL format
Rename *Abstract* to **prompt** and *ParaphrasedAbstract* to completion

In [ ]:
df_raw.rename(columns = {'Abstract':'prompt', 'ParaphrasedAbstract':'completion'}, inplace = True)

In [ ]:
#Taking only first 10 rows to save money
df=df_raw.head(10)
df.shape

(10, 2)

In [ ]:
df.to_json("paraphrasing_new.jsonl", orient='records', lines=True)

Using data prepartion tool provided by openai to fix any issues with our dataset. -q will auto accept all suggestions.

In [ ]:
!openai tools fine_tunes.prepare_data -f paraphrasing_new.jsonl -q

Analyzing...

- Your file contains 10 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- All completions end with suffix `.`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Add a suffix separator ` ->` 

Preparing test dataset

In [ ]:
df_test=df_raw.tail(3)
df_test.to_json("paraphrasing_new_test.jsonl", orient='records', lines=True)
!openai tools fine_tunes.prepare_data -f paraphrasing_new_test.jsonl -q

Analyzing...

- Your file contains 3 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- All prompts start with prefix `I`
- All completions end with suffix `.`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recomme

This tool made some corrections in existing file and saved it with a new name.  

##Fine-Tuning
Will use the new file created by data prepation tool to fine-tune ada model.
Can use following command if api key is stored in a variable or a file(OPENAI_API_SECRET is the api key):
!openai --api-key OPENAI_API_SECRET api fine_tunes.create -t "paraphrasing_new_prepared" -v "paraphrasing_new_test_prepared" -m ada

In [ ]:
!openai --api-key "" api fine_tunes.create -t "paraphrasing_new_prepared.jsonl" -v "paraphrasing_new_test_prepared.jsonl" -m ada

Upload progress: 100% 22.4k/22.4k [00:00<00:00, 41.5Mit/s]
Uploaded file from paraphrasing_new_prepared.jsonl: file-ghXpleojbafkD0E7TW8bVUn0
Upload progress: 100% 4.34k/4.34k [00:00<00:00, 8.09Mit/s]
Uploaded file from paraphrasing_new_test_prepared.jsonl: file-iWu5tGFiSjETc5aVwpI3wpiC
Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy



In [ ]:
!openai --api-key "" api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy

[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy



In [ ]:
!openai --api-key "" api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy

[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9
[2023-01-05 22:15:42] Fine-tune is in the queue. Queue number: 8

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy



In [ ]:
!openai --api-key "" api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy

[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9
[2023-01-05 22:15:42] Fine-tune is in the queue. Queue number: 8

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy



In [ ]:
!openai --api-key "" api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy

[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9
[2023-01-05 22:15:42] Fine-tune is in the queue. Queue number: 8
[2023-01-05 22:38:08] Fine-tune is in the queue. Queue number: 7
[2023-01-05 22:42:50] Fine-tune is in the queue. Queue number: 6

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy



In [ ]:
!openai --api-key "" api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy

[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9
[2023-01-05 22:15:42] Fine-tune is in the queue. Queue number: 8
[2023-01-05 22:38:08] Fine-tune is in the queue. Queue number: 7
[2023-01-05 22:42:50] Fine-tune is in the queue. Queue number: 6
[2023-01-05 22:45:40] Fine-tune is in the queue. Queue number: 5
[2023-01-05 22:49:08] Fine-tune is in the queue. Queue number: 4
[2023-01-05 22:49:52] Fine-tune is in the queue. Queue number: 3

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy



In [ ]:
!openai --api-key "" api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy

[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9
[2023-01-05 22:15:42] Fine-tune is in the queue. Queue number: 8
[2023-01-05 22:38:08] Fine-tune is in the queue. Queue number: 7
[2023-01-05 22:42:50] Fine-tune is in the queue. Queue number: 6
[2023-01-05 22:45:40] Fine-tune is in the queue. Queue number: 5
[2023-01-05 22:49:08] Fine-tune is in the queue. Queue number: 4
[2023-01-05 22:49:52] Fine-tune is in the queue. Queue number: 3
[2023-01-05 23:00:45] Fine-tune is in the queue. Queue number: 2
[2023-01-05 23:01:30] Fine-tune is in the queue. Queue number: 1
[2023-01-05 23:05:46] Fine-tune is in the queue. Queue number: 0

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy



In [ ]:
!openai --api-key "" api fine_tunes.follow -i ft-0gitoywCNU0H40CvBPvsvMJy

[2023-01-05 21:48:43] Created fine-tune: ft-0gitoywCNU0H40CvBPvsvMJy
[2023-01-05 21:49:50] Fine-tune costs $0.01
[2023-01-05 21:49:50] Fine-tune enqueued. Queue number: 10
[2023-01-05 21:50:39] Fine-tune is in the queue. Queue number: 9
[2023-01-05 22:15:42] Fine-tune is in the queue. Queue number: 8
[2023-01-05 22:38:08] Fine-tune is in the queue. Queue number: 7
[2023-01-05 22:42:50] Fine-tune is in the queue. Queue number: 6
[2023-01-05 22:45:40] Fine-tune is in the queue. Queue number: 5
[2023-01-05 22:49:08] Fine-tune is in the queue. Queue number: 4
[2023-01-05 22:49:52] Fine-tune is in the queue. Queue number: 3
[2023-01-05 23:00:45] Fine-tune is in the queue. Queue number: 2
[2023-01-05 23:01:30] Fine-tune is in the queue. Queue number: 1
[2023-01-05 23:05:46] Fine-tune is in the queue. Queue number: 0
[2023-01-05 23:07:46] Fine-tune started
[2023-01-05 23:08:03] Completed epoch 1/4
[2023-01-05 23:08:04] Completed epoch 2/4
[2023-01-05 23:08:06] Completed epoch 3/4
[2023-01-05 

## Fine-Tuned Model: ada:ft-personal-2023-01-05-23-08-28

##Results
A result file is attached to each job once it has been completed. This results file ID will be listed when you retrieve a fine-tune, and also when you look at the events on a fine-tune. Use fine tune job id to access this file.

In [ ]:
!openai --api-key "" api fine_tunes.results -i ft-0gitoywCNU0H40CvBPvsvMJy > result.csv

##Using this model on new data points
Using 2 datapoints from our raw dataset. Adjust token size to determine number of words in the result(paraphrased sentence).

In [ ]:
df_new_data=df_raw.iloc[20:22,]
df_new_data.to_json("new_datapoints.jsonl", orient='records', lines=True)
!openai tools fine_tunes.prepare_data -f new_datapoints.jsonl -q


Analyzing...

- Your file contains 2 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `.`
- All completions end with suffix `.`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Add a whitespace character to the beginning of the completion [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `new_datapoints_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "new_datapoints_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end wit

In [ ]:
new_data = pd.read_json('new_datapoints_prepared.jsonl', lines=True)
new_data.head()

,prompt,completion
0,Pre-trained language models have been shown to...,Pre-trained language models have been demonst...
1,Motion is an important cue for video predictio...,Motion is an important factor for video predi...


In [ ]:
new_data['prompt'][1]

'Motion is an important cue for video prediction and often utilized by separating video content into static and dynamic components. Most of the previous work utilizing motion is deterministic but there are stochastic methods that can model the inherent uncertainty of the future. Existing stochastic models either do not reason about motion explicitly or make limiting assumptions about the static part. In this paper, we reason about appearance and motion in the video stochastically by predicting the future based on the motion history. Explicit reasoning about motion without history already reaches the performance of current stochastic models. The motion history further improves the results by allowing to predict consistent dynamics several frames into the future. Our model performs comparably to the stateof-the-art models on the generic video prediction datasets, however, significantly outperforms them on two challenging real-world autonomous driving datasets with complex motion and dyna

In [ ]:
fine_tuned_model = 'ada:ft-personal-2023-01-05-23-08-28'
new_res = openai.Completion.create(model=fine_tuned_model, prompt=new_data['prompt'][1] + '\n\n###\n\n', max_tokens=40, temperature=0)
new_res['choices'][0]['text']

'\n\nThis paper is a stochastic model of motion in the video. It is based on the assumption that motion is deterministic. It is used to predict the future based on the motion history'

Using a sample text

In [ ]:
sample_abstract = """Multi-contrast MRI protocols increase the level of morphological information available for diagnosis. Yet, the number and quality of contrasts are limited in practice by various factors including scan time and patient motion. Synthesis of missing or corrupted contrasts from other high-quality ones can alleviate this limitation. When a single target contrast is of interest, common approaches for multi-contrast MRI involve either one-to-one or many-to-one synthesis methods depending on their input."""

In [ ]:
new_res = openai.Completion.create(model=fine_tuned_model, prompt=sample_abstract + '\n\n###\n\n', max_tokens=20, temperature=0)
new_res['choices'][0]['text']

'\n\nThe present study aimed to evaluate the performance of multi-contrast MRI in the detection of'